# Create Slice for Fabric Rack
This is the slice creation for the ```Ryu & Mininet``` tutorial. This is the first step to create the slice on fabric using fablib.

<b> Prerequisites  
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-5 at https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md#section-1-get-started.
 

## 1. Set up the Experiment

In this section you will use the Fablib manager to create a new slice that is composed of 2 nodes for the Ryu & Mininet testbed.
### 1.1 Reserve Resources
In this section we will declare the structure of the slice we will be using.

In [1]:
# Import Fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                     
fablib.show_config()
import json
import traceback

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,a70de2f5-9e12-4b6b-b412-0ae1a2c553b0
Bastion Host,bastion.fabric-testbed.net
Bastion Username,dabun01_0000229645
Bastion Private Key File,/home/fabric/work/fabric_config/fabric-bastion-key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Sites to avoid,


In [2]:
# Define and Submit Slice
slice_name = "MininetWorkspace"
site = "EDUKY"
print(site)
#The model of the VM
nicmodel = "NIC_Basic"
image = "default_ubuntu_20"

cores = 1
ram = 2
disk = 10

EDUKY


In [3]:
try:
    # Create a slice
    slice = fablib.new_slice(name=slice_name)
    #first Node
    ND1 = slice.add_node(name="Host", site=site)
    ND1.set_capacities(cores=cores, ram=ram, disk=disk)
    ND1.set_image(image)
    Net_ND1 = ND1.add_component(model=nicmodel, name="ND_Host").get_interfaces()[0]
    

    #Second Node
    ND2 = slice.add_node(name="Controller", site=site)
    ND2.set_capacities(cores=cores, ram=ram, disk=disk)
    ND2.set_image(image)
    Net_ND2 = ND2.add_component(model=nicmodel, name="ND_Controller").get_interfaces()[0] 

    #Lan
    LN = slice.add_l2network(name="Lan", interfaces=[Net_ND1, Net_ND2])
    
    # Submit the slice
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")


Retry: 10, Time: 241 sec


ID,3705b5eb-5a05-4001-9473-c31601894346
Name,MininetWorkspace
Lease Expiration (UTC),2024-12-05 22:44:57 +0000
Lease Start (UTC),2024-12-04 22:44:57 +0000
Project ID,a70de2f5-9e12-4b6b-b412-0ae1a2c553b0
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
e620bad4-a51e-4308-9167-029432d28338,Controller,1,2,10,default_ubuntu_20,qcow2,eduky-w5.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe94:a0a3,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe94:a0a3,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
87de401c-b4c3-471f-997b-ee5d8c9fee14,Host,1,2,10,default_ubuntu_20,qcow2,eduky-w10.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe43:f102,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe43:f102,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
b60d48dd-eead-4eaa-ada7-730208712dec,Lan,L2,L2Bridge,EDUKY,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Host-ND_Host-p1,p1,Host,Lan,100,config,,02:1E:95:D8:20:B4,enp7s0,enp7s0,fe80::1e:95ff:fed8:20b4,1,HundredGigE0/0/0/10
Controller-ND_Controller-p1,p1,Controller,Lan,100,config,,02:8A:2B:9F:0B:AA,enp7s0,enp7s0,fe80::8a:2bff:fe9f:baa,1,HundredGigE0/0/0/5



Time to print interfaces 249 seconds


### 1.2 Set up the Experiment Network
In this section of the slice creation, once the nodes are active, will assign the network spaces some predefined values that we will use for the assignment

In [4]:
# Setup Network
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    subnet = IPv4Network("10.0.0.0/24")
    Network = 'Lan'
    
    A = slice.get_node('Host')
    B = slice.get_node('Controller')
    
    #A's lan
    A_iface = A.get_interface(network_name=Network)  
    A_iface.ip_addr_add(addr='10.0.0.1', subnet=subnet)
    #B's Lan
    B_iface = B.get_interface(network_name=Network)  
    B_iface.ip_addr_add(addr='10.0.0.2', subnet=subnet)
    
    for node in slice.get_nodes():
        for interface in node.get_interfaces():
            stdout, stderr = node.execute(f'sudo ip link set dev { interface.get_device_name()} up')
            if (node.get_name() == "ND_A"):
                A_Device_Name = interface.get_device_name()
        print(f"Network status at node {node.get_name()}")
        node.execute("ip a")
        
except Exception as e:
    print(f"Fail: {e}")

Network status at node Host
1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: enp3s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 9000 qdisc fq_codel state UP group default qlen 1000
    link/ether fa:16:3e:43:f1:02 brd ff:ff:ff:ff:ff:ff
    inet 10.30.8.214/19 brd 10.30.31.255 scope global dynamic enp3s0
       valid_lft 86028sec preferred_lft 86028sec
    inet6 2610:1e0:1700:206:f816:3eff:fe43:f102/64 scope global dynamic mngtmpaddr noprefixroute 
       valid_lft 86324sec preferred_lft 14324sec
    inet6 fe80::f816:3eff:fe43:f102/64 scope link 
       valid_lft forever preferred_lft forever
3: enp7s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:1e:95:d8:20:b4 brd ff:ff:ff:f

### Test the connection for the nodes
This code will ping each respective node


In [5]:
 # Test connectivity between Host_1 and ControllerNode
print("Pinging ControllerNode from Host_1...")
A.execute('ping -c 4 10.0.0.2')  # Ping ControllerNode from Host_1
    
print("Pinging Host_1 from ControllerNode...")
B.execute('ping -c 4 10.0.0.1')  # Ping Host_1 from ControllerNode

Pinging ControllerNode from Host_1...
PING 10.0.0.2 (10.0.0.2) 56(84) bytes of data.
64 bytes from 10.0.0.2: icmp_seq=1 ttl=64 time=0.211 ms
64 bytes from 10.0.0.2: icmp_seq=2 ttl=64 time=0.098 ms
64 bytes from 10.0.0.2: icmp_seq=3 ttl=64 time=0.105 ms
64 bytes from 10.0.0.2: icmp_seq=4 ttl=64 time=0.092 ms

--- 10.0.0.2 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3074ms
rtt min/avg/max/mdev = 0.092/0.126/0.211/0.049 ms
Pinging Host_1 from ControllerNode...
PING 10.0.0.1 (10.0.0.1) 56(84) bytes of data.
64 bytes from 10.0.0.1: icmp_seq=1 ttl=64 time=0.110 ms
64 bytes from 10.0.0.1: icmp_seq=2 ttl=64 time=0.077 ms
64 bytes from 10.0.0.1: icmp_seq=3 ttl=64 time=0.088 ms
64 bytes from 10.0.0.1: icmp_seq=4 ttl=64 time=0.109 ms

--- 10.0.0.1 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3068ms
rtt min/avg/max/mdev = 0.077/0.096/0.110/0.014 ms


('PING 10.0.0.1 (10.0.0.1) 56(84) bytes of data.\n64 bytes from 10.0.0.1: icmp_seq=1 ttl=64 time=0.110 ms\n64 bytes from 10.0.0.1: icmp_seq=2 ttl=64 time=0.077 ms\n64 bytes from 10.0.0.1: icmp_seq=3 ttl=64 time=0.088 ms\n64 bytes from 10.0.0.1: icmp_seq=4 ttl=64 time=0.109 ms\n\n--- 10.0.0.1 ping statistics ---\n4 packets transmitted, 4 received, 0% packet loss, time 3068ms\nrtt min/avg/max/mdev = 0.077/0.096/0.110/0.014 ms\n',
 '')

### 1.3 Configure the software needed for the nodes in the experiment
This section of the slice creation, we will add any aditional software, tools or scripts that we need for our experiments.

In [6]:
# Install Software including Mininet on Host
try:
    # Install common software on all nodes
    for node in slice.get_nodes():
        node.execute('yes | sudo apt-get -y update && sudo apt-get upgrade -y') 
        node.execute('sudo apt-get -y install net-tools')

    # Install Ryu controller and dependencies on the Controller node
    B.execute('sudo apt install python3-pip -y')
    B.execute('sudo apt install gcc python-dev libffi-dev libssl-dev libxml2-dev libxslt1-dev zlib1g-dev -y')
    B.execute('sudo apt-get install python3-ryu -y')
    B.execute('git clone https://github.com/faucetsdn/ryu.git')

    # Install Mininet on Host_1
    print("Installing Mininet on Host_1...")
    A.execute('sudo apt-get -y install mininet')
    
    print("Software installation completed successfully.")

except Exception as e:
    print(f"Fail: {e}")
    

Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease [265 kB]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe Translation-en [5124 kB]
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 c-n-f Metadata [265 kB]
Get:8 http://nova.clouds.archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [144 kB]
Get:9 http://nova.clouds.archive.ubuntu.com/ubuntu focal/multiverse Translation-en [104 kB]
Get:10 http://nova.clouds.archive.ubuntu.com/ubuntu focal/multiverse amd64 c-n-f Metadata [9136 B]
Get:11 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3683 kB]
Get:12 http://nova.clou

## Continue to The experiment notebook

Once you have completed this notebook you should be able to continue to the Ryu & Mininet Notebook. You can either open it on the explorer or click [Here](./Mininet.ipynb) to open the next notebook.